# 5. Annotate cell clusters

This notebook compiles cell cluster annotations from the Cao et al. 2019 analysis to generate cluster names and tissue colors for the Piekarz et al. 2024 scRNAseq data.

## 5.1 Setup

Load the necessary libraries and set the data directory.

In [1]:
from pathlib import Path

import zoogletools as zt

/Users/dennis/miniconda3/envs/2025-zoogle/lib/python3.12/site-packages/anndata/utils.py:434: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/Users/dennis/miniconda3/envs/2025-zoogle/lib/python3.12/site-packages/anndata/utils.py:434: FutureWarning: Importing read_text from `anndata` is deprecated. Import anndata.io.read_text instead.
  warnings.warn(msg, FutureWarning)
/Users/dennis/miniconda3/envs/2025-zoogle/lib/python3.12/site-packages/anndata/utils.py:434: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  warnings.warn(msg, FutureWarning)
/Users/dennis/miniconda3/envs/2025-zoogle/lib/python3.12/site-packages/anndata/utils.py:434: FutureWarning: Importing read_mtx from `anndata` is deprecated. Import anndata.io.read_mtx instead.
  warnings.warn(msg, FutureWarning)
/Users/dennis/miniconda3/envs/2025-zoogle/lib/python3.12/site-packages/anndata

## 5.2 Load cell clusters

Load the cell clusters from the Cao et al. 2019 analysis.
This also processes the data to generate a merging barcode for each cell cluster for combining with the Piekarz et al. 2024 scRNAseq data.

In [2]:
cell_clusters = zt.ciona.data_processing.load_cell_clusters()
display(cell_clusters)

,NAME,X,Y,Clusters,Tissue Type,Cao_stage,stage,replicate,stage_replicate,barcode,merging_barcode
1,C110.1_AAACCTGTCAGTTTGG,1.771697179,0.169352728,44,epidermis,C110,iniG,rep1,iniG_rep1,AAACCTGTCAGTTTGG,iniG_rep1_AAACCTGTCAGTTTGG
2,C110.1_AAACGGGTCTGTCCGT,-6.462542321,6.657707053,14,epidermis,C110,iniG,rep1,iniG_rep1,AAACGGGTCTGTCCGT,iniG_rep1_AAACGGGTCTGTCCGT
3,C110.1_AAAGATGAGTTGAGAT,-2.589877535,-1.603552418,75,epidermis,C110,iniG,rep1,iniG_rep1,AAAGATGAGTTGAGAT,iniG_rep1_AAAGATGAGTTGAGAT
4,C110.1_AAATGCCGTCGCATAT,-12.31563402,-6.27978969,44,epidermis,C110,iniG,rep1,iniG_rep1,AAATGCCGTCGCATAT,iniG_rep1_AAATGCCGTCGCATAT
5,C110.1_AAATGCCGTGTTTGGT,-2.731373449,-1.449560153,75,epidermis,C110,iniG,rep1,iniG_rep1,AAATGCCGTGTTTGGT,iniG_rep1_AAATGCCGTGTTTGGT
...,...,...,...,...,...,...,...,...,...,...,...
90575,lv.1_TCGTAGAGTACCGAGA-2,29.1525446,-6.089481097,89,endoderm,lv,larva,rep1,larva_rep1,TCGTAGAGTACCGAGA-2,larva_rep1_TCGTAGAGTACCGAGA-2
90576,lv.1_TGAGAGGCACGAGAGT-2,2.836903468,-27.77839935,67,nervous system,lv,larva,rep1,larva_rep1,TGAGAGGCACGAGAGT-2,larva_rep1_TGAGAGGCACGAGAGT-2
90577,lv.1_TGGGCGTTCGCCTGAG-2,1.730967885,-28.28614185,67,nervous system,lv,larva,rep1,larva_rep1,TGGGCGTTCGCCTGAG-2,larva_rep1_TGGGCGTTCGCCTGAG-2
90578,lv.1_TGTTCCGAGTGTCCAT-2,3.113585251,-27.65757074,67,nervous system,lv,larva,rep1,larva_rep1,TGTTCCGAGTGTCCAT-2,larva_rep1_TGTTCCGAGTGTCCAT-2


## 5.3 Quantify cluster annotations

Quantify the cluster annotations from the Cao et al. 2019 analysis. This function takes the Cao et al. 2019 cell cluster annotations and a directory containing the scRNAseq data. It then quantifies the cluster annotations by counting the number of cells in each cluster for each stage.

The output is one TSV file per stage with the counts of cells of each type in each cluster.

In [3]:
input_dirpath = Path("../../data/Ciona_intestinalis_scRNAseq_data_Piekarz")
output_dirpath = Path("../../data/Ciona_intestinalis_scRNAseq_data_Piekarz/cluster_annotations")

zt.ciona.cluster_annotations.quantify_cluster_annotations(
    cell_clusters, input_dirpath=input_dirpath, output_dirpath=output_dirpath
)

## 5.4 Compile cluster names

Compile the cluster names from the Cao et al. 2019 analysis.

This function takes the output directory from the `quantify_cluster_annotations` function and a path to save the compiled cluster names.

The output is a TSV file with the formatted cluster names and the tissue colors.

Cluster names take the format `<top_cell_type>_<tissue_type_suffix>(<fraction_tissue_type>)`. For example, `epidermis_2(0.5)`.
- `<top_cell_type>` is the most abundant cell type in the cluster.
- `<tissue_type_suffix>` is a suffix indicating the number of clusters with that cell type. For example, if there are 2 clusters with the same cell type, the first cluster will have a suffix of `_1` and the second cluster will have a suffix of `_2`. Tissue types with only one cluster are not given a suffix.
- `<fraction_tissue_type>` is the fraction of cells in the cluster that are of the tissue type.

If the second most-abundant tissue type has at least half the cells of the most-abundant tissue type, then the cluster name will also include the fraction of cells of the second most-abundant tissue type.
For example, `epidermis_2(0.5)+nervous-system(0.4)` indicates that the cluster contains 50% epidermis and 40% nervous system.

In [4]:
zt.ciona.cluster_annotations.process_all_quantified_cluster_annotations(
    output_dirpath, output_dirpath / "Ciona_scRNAseq_cluster_names.tsv"
)

TypeError: first argument must be callable or None

## 5.5 Illustrating color palette for the tissue types

This cell shows the color palette for each of the tissue types.

In [5]:
zt.ciona.cluster_annotations.TISSUE_PALETTE_DICT

{'epidermis':                      
    dusk   #094468 0.0
    lapis  #2B65A1 0.234
    aegean #5088C5 0.46
    vital  #73B5E3 0.711
    sky    #C6E7F4 1.0,
 'nervous-system':                      
    concord   #341E60 0.0
    tanzanite #54448C 0.22
    aster     #7A77AB 0.489
    wish      #BABEE0 0.85
    iris      #DCDFEF 1.0,
 'notochord':                      
    mud     #635C5A 0.0
    bark    #8F8885 0.356
    chateau #BAB0A8 0.655
    taupe   #DBD1C3 0.874
    stone   #EDE6DA 1.0,
 'mesenchyme':                      
    cinnabar  #9E3F41 0.0
    dragon    #C85152 0.251
    amber     #F28360 0.604
    tangerine #FFB984 0.883
    melon     #FFD0B0 1.0,
 'muscle-heart':                      
    azalea #C14C70 0.0
    candy  #E2718F 0.318
    rose   #F898AE 0.595
    dress  #F8C5C1 0.824
    putty  #FFE3D4 1.0,
 'endoderm':                      
    umber   #A85E28 0.0
    mustard #D68D22 0.416
    canary  #F7B846 0.745
    sun     #FFD364 0.908
    oat     #F5E4BE 1.0,
 'unann